In [ ]:
'''

  Вариант 69

'''
import pandas as pd
import sqlite3

In [ ]:
conn = sqlite3.connect('store.sqlite')

with open('store.db', 'r', encoding='utf-8') as dump:
  script_dump = dump.read()

conn.executescript(script_dump)
conn.commit()

In [ ]:
'''

  Задание 1:

  Вывести информацию о книгах,
  которые относятся или к жанру Роман, или к жанру Приключения,
  а их цена или меньше 400 рублей, или больше 500.
  В выборку включить название книги, фамилию автора, название жанра,
  количество экземпляров и цену книги.
  Информацию отсортировать сначала по названию книги в алфавитном порядке,
  а затем по фамилии автора тоже в алфавитном порядке.

'''

pd.read_sql(
    '''
        SELECT
            title,
            name_author,
            name_genre,
            amount,
            price
        FROM
            genre
            JOIN book USING (genre_id)
            JOIN author USING (author_id)
        WHERE
            price < 400
            OR price > 500
        ORDER BY
            title,
            name_author
    ''',
    conn
)

,title,name_author,name_genre,amount,price
0,Белая гвардия,Булгаков М.А.,Роман,5,540.50
1,Братья Карамазовы,Достоевский Ф.М.,Роман,3,799.01
2,Лирика,Пастернак Б.Л.,Поэзия,2,518.99
3,Мастер и Маргарита,Булгаков М.А.,Роман,3,670.99
4,Стихотворения и поэмы,Есенин С.А.,Поэзия,15,650.00
5,Черный человек,Есенин С.А.,Поэзия,6,570.20


In [ ]:
'''

  Задание 2:

  Для каждого читателя вывести количество различных книг,
  которые он брал в библиотеке. Если читатель никаких книг не брал - вывести 0.
  Столбцы назвать Читатель, Количество_книг.
  Информацию отсортировать сначала по убыванию количества книг,
  а потом по фамилии читателя в алфавитном порядке.

'''

pd.read_sql(
    '''
        WITH cte(name_client, book_count) AS (
            SELECT
                name_client,
                COUNT(book_id)
            FROM
                client
                LEFT JOIN buy USING (client_id)
                LEFT JOIN buy_book USING (buy_id)
            GROUP BY
                client_id
        )

        SELECT
            name_client AS Читатель,
            book_count AS Количество_книг
        FROM
            cte
        ORDER BY
            book_count DESC,
            name_client ASC
    ''',
    conn
)

,Читатель,Количество_книг
0,Баранов Павел,4
1,Абрамова Катя,3
2,Семенонов Иван,1
3,Яковлева Галина,0


In [ ]:
'''

  Задание 3:

  Вывести книги тех авторов,
  книги которых чаще всего заказывали покупатели.

'''

pd.read_sql(
    '''
        WITH get_max_count(max_count) AS (
            SELECT
                COUNT(book_id)
            FROM
                author
                JOIN book USING (author_id)
                JOIN buy_book USING (book_id)
            GROUP BY
                author_id
            ORDER BY
                COUNT(book_id) DESC
            LIMIT 1
        ),
        get_authors_id (author_id) AS (
            SELECT
                author_id
            FROM
                author
                JOIN book USING (author_id)
                JOIN buy_book USING (book_id),
                get_max_count
            GROUP BY
                author_id
            HAVING
                COUNT(book_id) = max_count
        )

        SELECT
            *
        FROM
            get_authors_id
            JOIN book USING (author_id)
            JOIN author USING (author_id)
    ''',
    conn
)

,author_id,book_id,title,genre_id,price,amount,name_author
0,1,1,Мастер и Маргарита,1,670.99,3,Булгаков М.А.
1,1,2,Белая гвардия,1,540.50,5,Булгаков М.А.
2,2,3,Идиот,1,460.00,10,Достоевский Ф.М.
3,2,4,Братья Карамазовы,1,799.01,3,Достоевский Ф.М.
4,2,5,Игрок,1,480.50,10,Достоевский Ф.М.


In [ ]:
'''

  Задание 4:

  Создать таблицу good_order, в которую включить книги и их авторов,
  а также количество экземпляров книг, которое нужно заказать.
  Количество экземпляров указать такое, чтобы на складе
  (после получения книг) было одинаковое количество книг каждого автора,
  равное максимальному количеству книг этого автора. Последний столбец назвать Заказ.
  Если количество заказываемой книги равно 0, эту книгу в таблицу не включать

'''

conn.executescript(
    '''
        DROP TABLE IF EXISTS good_order;

        CREATE TABLE good_order (
            good_order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name_author VARCHAR(50),
            title VARCHAR(50),
            Заказ INTEGER
        );

        INSERT INTO good_order (name_author, title, Заказ)
        WITH author_max_amount (author_id, max_amount) AS (
            SELECT
                author_id,
                MAX(amount)
            FROM
                author
                JOIN book USING (author_id)
            GROUP BY
                author_id
        )
        SELECT
            name_author,
            title,
            max_amount - amount
        FROM
            author
            JOIN author_max_amount USING (author_id)
            JOIN book USING (author_id)
        WHERE
            0 < (max_amount - amount)
    '''
)

In [ ]:
pd.read_sql(
    '''
      SELECT * FROM good_order
    ''',
    conn
)

,good_order_id,name_author,title,Заказ
0,1,Булгаков М.А.,Мастер и Маргарита,2
1,2,Достоевский Ф.М.,Братья Карамазовы,7
2,3,Есенин С.А.,Черный человек,9


In [ ]:
'''

  Задание 5:

  В магазине решили продавать книги одного автора в комплекте,
  состоящем из двух книг.
  Для этого отсортировали книги каждого автора по убыванию цены
  и в комплект к каждой книге включили следующую по цене.

'''

pd.read_sql(
    '''
        WITH cte (name_author, first_title, second_title, first_price, second_price)
        AS (
             SELECT
                name_author,
                title,
                COALESCE(
                  LEAD(title) OVER (ORDER BY price DESC),
                  FIRST_VALUE(title) OVER (ORDER BY price DESC)
                ) as next_title,
                price,
                COALESCE(
                  LEAD(price) OVER (ORDER BY price DESC),
                  FIRST_VALUE(price) OVER (ORDER BY price DESC)
                ) as next_price
            FROM
                author
                JOIN book USING (author_id)
            ORDER BY
                price DESC
        )
        SELECT
            name_author AS Автор,
            first_title || ". " || second_title AS Книги,
            ROUND((first_price + second_price) * 75 / 100, 2) AS Цена
        FROM
            cte
    ''',
    conn
)

,Автор,Книги,Цена
0,Достоевский Ф.М.,Братья Карамазовы. Мастер и Маргарита,1102.50
1,Булгаков М.А.,Мастер и Маргарита. Стихотворения и поэмы,990.74
2,Есенин С.А.,Стихотворения и поэмы. Черный человек,915.15
3,Есенин С.А.,Черный человек. Белая гвардия,833.03
4,Булгаков М.А.,Белая гвардия. Лирика,794.62
5,Пастернак Б.Л.,Лирика. Игрок,749.62
6,Достоевский Ф.М.,Игрок. Идиот,705.38
7,Достоевский Ф.М.,Идиот. Братья Карамазовы,944.26
